In [105]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Set Up Environment

In [215]:
network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210106025209.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://52.53.175.151:9000/",       "nid": 3}}

env = connections[network]

In [216]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [217]:
contracts = {}

# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': ''},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': ''},
 'dividends': {'zip': 'core_contracts/staking.zip',
  'SCORE': ''},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': ''},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': ''},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': ''},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''},
 'reserve_fund': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': ''},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': ''}}

In [218]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [219]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(9000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service2.send_transaction(signed_transaction)
tx_hash

'0x23fd8199d269d46e5dd3e2ad3a451fec7e5c6d36225304ea3917a8ebc58411ac'

In [220]:
wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")

In [164]:
# Balanced test wallet
btest_wallet = KeyWallet.load("keystores/balanced_test.json", "xxxx")

### Send ICX to Balanced test wallet

In [221]:
test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

'0xd6b90bfb1e06a51ac1d239c69a41f454ab3eb359f0bb928998d6ba1db564ec03'

In [ ]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# contracts['dividends'] = {'zip': 'core_contracts/dividends.zip', 'SCORE': 'cxceadb6419b92f7170b38b8328dbd8deefb4a1cdb'}
deploy = ['loans', 'staking', 'dividends', 'sicx', 'dex', 'icd', 'dummy_oracle', 'reserve_fund', 'bal']
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

## Deploy All SCOREs

In [250]:
step_limit = 4000100000

for score, loc in contracts.items():
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(res)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


...

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


reserve_fund
{'txHash': '0xb310749a1b7ee9c3277e16ba29f64ceab79e981a090cc52b8e07f5770effdee1', 'blockHeight': 261074, 'blockHash': '0xdecb1c21cfc5f9e4ece55d706a38cfc44eb909c7ca0f99953e1a8db8ab1ac1dd', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx3eba3b888495c6ba7e4b18af8c898f615e02f3d2', 'stepUsed': 1168860000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1168860000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxb12ac82038f98353776f555b60c7ade39bc73681'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxd361228ccf434754824fc04b2ce7dc05becac3b9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx1631edf6027bb89ef99a613d83969239c2040516'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx16bc08d749b028ba330a6071c66f9b9c34b9fc85'},
 'reserve_fund': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx3eba3b888495c6ba7e4b18af8c898f615e02f3d2'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxba42914c86964530e5c52e142c8862dc6c4a4e12'}}

### Deploy a single SCORE

In [140]:
# Deploy a single SCORE
score = 'bal'
loc = {'zip': 'token_contracts/bal.zip',  'SCORE': ''}
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(loc["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[score]["SCORE"] = res.get('scoreAddress', '')
res

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


{'txHash': '0xee2e67871531096d1b8793d571fc8e28fade7871a3a59373794df48bd18948fb',
 'blockHeight': 240365,
 'blockHash': '0x1655134496e9ad1fe80163abf08821a3a9abf39ee3df88d3f515bf3ad247d78f',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cxa3ee97d35281050ae626e6b6b9d9e35d3e39cc03',
 'stepUsed': 1180235680,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1180235680,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Update a SCORE

In [265]:
contract = 'loans'
update = contracts[contract]['SCORE']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contracts[contract]["zip"]))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...


{'txHash': '0x066d4cce6502bf9da2fb34df5c433e43638d5869f5f269808ac125f6e4fc76c8',
 'blockHeight': 262728,
 'blockHash': '0x0e6d6316416bcf2cdc3dd3ca2d86429996b3a54fa40135ce7c95b25b2ed50bbc',
 'txIndex': 1,
 'to': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f',
 'scoreAddress': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f',
 'stepUsed': 2433903380,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2433903380,
 'eventLogs': [{'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
    '0xad78ebc5ac6200000'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d',
   'indexed': ['Mint(Address,int,bytes)',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be'],
   'data': ['0xad78ebc5ac6200000', '0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x

## Set parameters

In [251]:
settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'icd', 'method': 'setAdmin', 'params':{'_admin': contracts['loans']['SCORE']}},
            {'contract': 'bal', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'loans', 'method': 'setAdmin', 'params':{'_admin': wallet.get_address()}},
            {'contract': 'icd', 'method': 'set_oracle_address', 'params':{'_address': contracts['dummy_oracle']['SCORE']}},
            {'contract': 'staking', 'method': 'set_sICX_address', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'loans', 'method': 'set_dividends', 'params':{'_address': contracts['dividends']['SCORE']}},
            {'contract': 'loans', 'method': 'set_reserve', 'params':{'_address': contracts['reserve_fund']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_loans_score', 'params':{'_address': contracts['loans']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_baln_token', 'params':{'_address': contracts['bal']['SCORE']}},
            {'contract': 'reserve_fund', 'method': 'set_sicx_token', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'sicx', 'method': 'set_staking_address', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'bal', 'method': 'mint', 'params':{'_amount': 10 * ICX}},
            {'contract': 'bal', 'method': 'mintTo', 'params':{'_account': contracts['reserve_fund']['SCORE'], '_amount': 1000 * ICX}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['sicx']['SCORE'], 'is_active': 1, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['icd']['SCORE'], 'is_active': 1, 'is_collateral': 0}},
            {'contract': 'loans', 'method': 'add_asset', 'params':{'_token_address': contracts['bal']['SCORE'], 'is_active': 0, 'is_collateral': 1}},
            {'contract': 'loans', 'method': 'toggle_loans_on', 'params':{}}]

In [252]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setAdmin, with parameters {'_admin': 'cxb12ac82038f98353776f555b60c7ade39bc73681'} on the sicx contract.


'0xd24198659eea399ee0895c2b80081cc7240ebd9fe07f224d7dbece6d960b1d7d'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0xd24198659eea399ee0895c2b80081cc7240ebd9fe07f224d7dbece6d960b1d7d', 'blockHeight': 261082, 'blockHash': '0x347aede3918bc957520d47e0da28e736114cb54e13cb269a0b7c3ac02e38357a', 'txIndex': 1, 'to': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'stepUsed': 148920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148920, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

'0x8e65b2e45534cc000622409f9f66c57b51fa92c8b153168fcc4abad2833ee119'

...

'0xd71054c2e17dfbd90ad068bf74a069ad2f5f38556ba533622ce04b143590817c'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0xd71054c2e17dfbd90ad068bf74a069ad2f5f38556ba533622ce04b143590817c', 'blockHeight': 261100, 'blockHash': '0xade9d5e08bf904b2f51cc60c064b695a192aee6905d8e793a0fca8115fab84f7', 'txIndex': 1, 'to': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'stepUsed': 130680, 'stepPrice': 10000000000, 'cumulativeStepUsed': 130680, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Run through read-only methods on all SCOREs

### A single read-only method

In [ ]:
params = {'_owner': wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [240]:
ro_methods = {"loans": ["name", "get_available_assets", "asset_count", "get_total_collateral"],
              "staking": ["name", "get_sICX_address", "get_rate"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [268]:
results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result


SCORE: loans
Address: cx60ba0aa664572602b2667d6d6d4b217a36082d0f
name: BalancedLoans
get_available_assets: {'sICX': {'symbol': 'sICX', 'address': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'peg': 'sICX', 'added': '0x5b8abdb57ed07', 'is_collateral': '0x1', 'active': '0x1'}, 'ICD': {'symbol': 'ICD', 'address': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'peg': 'USD', 'added': '0x5b8abdb7672ea', 'is_collateral': '0x0', 'active': '0x1', 'bad_debt': '0x15af1d78b58c400000', 'liquidation_pool': '0x248f1c2fa4e3087a56096329820000000'}}
asset_count: 0x3
get_total_collateral: 1602.5244477

SCORE: staking
Address: cxb12ac82038f98353776f555b60c7ade39bc73681
name: Staking
get_sICX_address: cxc92b0adc5f889044642ac3c1d24842eae5d222b4
get_rate: 1.0

SCORE: sicx
Address: cxc92b0adc5f889044642ac3c1d24842eae5d222b4
name: StakedICX
symbol: sICX
decimals: 18
totalSupply: 1612.769
getAdmin: cxb12ac82038f98353776f555b60c7ade39bc73681

SCORE: icd
Address: cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d
name: 

## Deposit ICX and Mint ICD Loan

In [254]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0x7a59c521551a2b9cca99503a59d15c83aea68506039406cd1724c2970acf4e6e'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0x7a59c521551a2b9cca99503a59d15c83aea68506039406cd1724c2970acf4e6e', 'blockHeight': 261110, 'blockHash': '0xc9259f8279b06e372388f3edd71e2631ce243902b8aa9af99d2f96fae25c0992', 'txIndex': 1, 'to': 'cxb12ac82038f98353776f555b60c7ade39bc73681', 'stepUsed': 596060, 'stepPrice': 10000000000, 'cumulativeStepUsed': 596060, 'eventLogs': [{'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['OraclePrice(str,int,int)', 'ICXUSD', '0x2454d8dcb2cf44bb', '0x5b8abdce4ed41'], 'data': []}, {'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x22b1c8c1227a00000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x22b1c8c1227a00000', '0x4e6f6e65']}, {'scoreAddre

### Just deposit collateral

In [256]:
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0xb93add627a791a8a9265b36e35485cc652892255dc142cd879d353f87f740229'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0xb93add627a791a8a9265b36e35485cc652892255dc142cd879d353f87f740229', 'blockHeight': 261127, 'blockHash': '0x701ca86db69a51950f458b926ae24691279195e48f558bb70fd6158463dda4a7', 'txIndex': 1, 'to': 'cxb12ac82038f98353776f555b60c7ade39bc73681', 'stepUsed': 338900, 'stepPrice': 10000000000, 'cumulativeStepUsed': 338900, 'eventLogs': [{'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', '0x1a055690d9db80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Mint(Address,int,bytes)', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f'], 'data': ['0x1a055690d9db80000', '0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f7

### Repay Some ICD

In [257]:
data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0x02dd6146ca73c3b209f22a609a4dab386136cfa839a4f03f0034bd7031721897'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x02dd6146ca73c3b209f22a609a4dab386136cfa839a4f03f0034bd7031721897', 'blockHeight': 261130, 'blockHash': '0x2b95375fa77a59fafb5f217e0d92d907024135c9588431edeeb90d09331e803d', 'txIndex': 1, 'to': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'stepUsed': 328720, 'stepPrice': 10000000000, 'cumulativeStepUsed': 328720, 'eventLogs': [{'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['Burn(Address,int)', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f'], 'data': ['0x4563918244f40000']}, {'scoreAddress': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'ICD', '0x4563918244f40000'], 'data': ['Loan of 5.0 ICD repaid to Balanced.']}, {'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', '0x4563918244f

### Withdraw Some Collateral

In [258]:
params = {'_value': 5 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdraw_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0xee2fb3554103807fe9ac94b6438dc3db60a7e212d0121b3f274b78324ef5e7e0'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0xee2fb3554103807fe9ac94b6438dc3db60a7e212d0121b3f274b78324ef5e7e0', 'blockHeight': 261133, 'blockHash': '0xe5b4d5f081dda7cbc460d02f4ff3b63189bac61fcb5e7e95d9d4c7ced760ef93', 'txIndex': 1, 'to': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'stepUsed': 272700, 'stepPrice': 10000000000, 'cumulativeStepUsed': 272700, 'eventLogs': [{'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['OraclePrice(str,int,int)', 'ICXUSD', '0x2454d8dcb2cf44bb', '0x5b8abdfa363a4'], 'data': []}, {'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x4563918244f40000'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x4563918244f40000'], 'data': ['Collateral withdrawn. 

### Test Liquidation

In [259]:
# Deposit collateral to fresh wallet
data = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add_collateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0x9b66cea220d0024d5f39227be905b667a250989c9c5902cefe43df9a7e565ba0'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...
{'txHash': '0x9b66cea220d0024d5f39227be905b667a250989c9c5902cefe43df9a7e565ba0', 'blockHeight': 261140, 'blockHash': '0x8180f5f4fbc8065be3c1dec73135b5ab8c8925f8e14cfce066ba36a91358ab74', 'txIndex': 1, 'to': 'cxb12ac82038f98353776f555b60c7ade39bc73681', 'stepUsed': 349920, 'stepPrice': 10000000000, 'cumulativeStepUsed': 349920, 'eventLogs': [{'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', '0x2a6f1a22364bbe8000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']}, {'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Mint(Address,int,bytes)', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f'], 'data': ['0x2a6f1a22364bbe8000', '0x7b226d6574686f64223a20225f6465706f7369745f616e645f626

#### Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [266]:
params = {'_position': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash
print(get_tx_result(tx_hash))

'0x6c5e6ef87337ceb60289fb90918cfdc56c6f9c184267b0e733f624b9fe158bad'

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Retrying in 2 seconds...
{'txHash': '0x6c5e6ef87337ceb60289fb90918cfdc56c6f9c184267b0e733f624b9fe158bad', 'blockHeight': 262739, 'blockHash': '0x20391ec93ad6b5cc65c5fc84e4a80c9567c14ef3662cc18c1b81baccf3668552', 'txIndex': 1, 'to': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'stepUsed': 348100, 'stepPrice': 10000000000, 'cumulativeStepUsed': 348100, 'eventLogs': [{'scoreAddress': 'cxa7ef2f228c1d3ee6bab8595245324a10e2698d8d', 'indexed': ['OraclePrice(str,int,int)', 'ICXUSD', '0x2454d8dcb2cf44bb', '0x5b8ac9f41ae79'], 'data': []}, {'scoreAddress': 'cxc92b0adc5f889044642ac3c1d24842eae5d222b4', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x48c86487af753800'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx60ba0aa664572602b2667d6d6d4b217a36082d0f', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x48c86487af753800'], 'data': ['Liquidation reward of 

In [267]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'created': '0x5b8abe07927b5',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'assets': {'sICX': [0, 0], 'ICD': [0, 0]},
 'standing': 'Liquidate',
 'updated': '0x5b8ac9f41ae79'}

In [263]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('get_account_positions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'created': '0x5b8abe07927b5',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'assets': {'sICX': [782769000000000000000, 782769000000000000000],
  'ICD': [200000000000000000000, 200000000000000000000]},
 'standing': 'Locked',
 'updated': '0x5b8abe59a9dbb'}

In [179]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['icd']['SCORE'])\
                    .method('balanceOf')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

'0xad78ebc5ac6200000'